In [ ]:
# Tracking rainforest development in the amazon rainforest between 2010 and 2020

Within this project we are trying to show and calculate forest loss in the South American Amazon rainforest within a ten year period. We will use the land cover data cube 
(https://s3.eu-central-1.amazonaws.com/deep-esdl-public/LC-1x2160x2160-1.0.0.levels/5.zarr) that relies on data provided and summarised by the Université catholique de Louvain.
It is widely known that the amazon rainforest decreases by a large amount of trees every year, but we wanted to see if it is possible to see that large decrease within the 
ten years of the data cube as well. As a tool to develope our model, we created three modules to process the data.
sample cube, extend, sources

A few things are to be noted before putting the results into perspective: An error of about 5 % was proclaimed for the data set.
The way the methodology to created the data was designed, not all changed between different LCCS categories are sometimes detected, putting the results into a different perspective.
It is also worth nothing that abrupt changes are better captured than gradual ones. Since the deforestation is a relatively drastic event into the landscape, one should think that it will definitely showing in the data. The base of the maps that created the data cube is a 300m spatial resolution, which was calculated off 1km pixels, meaning any changes under 1km will not be displayed.
Also changes occurring in the 2014 - 2015 period are reduced to forest changes only. It is not further specified if that includes only loss of forest or gain of forest as well.
In the category that we used for the detection of rainforest (tree_broadleaved_evergreen_closed_to_open), if the amount of broadleaved, evergreen forest in that pixel is betweek 16 and 100%, it will be classified into that category. It is therefore possible that in an "tree_broadleaved_evergreen_closed_to_open" pixel, only 20% are actually broadleaved evergreen trees. Gaps were filled with nearest-neighbour method that has its own positive and negative aspects.
The total tree fraction of a pixel that has >0% trees in it is always assigned to a single tree type. In our class used it means that the >15% broadleaf, evergreen trees will get the broad-leaved evergreen tree type. 90% in that class group are actually broadleaved-evergreen trees while 10% are natural grasses.


In [ ]:
using Pkg
Pkg.activate(".")
#Pkg.resolve()
#Pkg.instantiate()
using YAXArrays
using Zarr
using Dates

In [ ]:
lc = Rainforestlib.get_lcc_datacube()

filtered_dc = Rainforestlib.rough_spatial_filter(lc; time_bounds = (Date(2010), Date(2021)))

In [ ]:
include("src/LCCSClasses.jl")
using .LCCSClasses

# create a set of accepted values
rainforest_values = Set(LCCSClasses.categories["rainforest"].lccs_classes)
forest_values = Set(LCCSClasses.categories["forest"].lccs_classes)
all_forest_values = union(forest_values, rainforest_values)

In [ ]:
# WARNING: do not try this with the huge datacube, it wont work. The in function needs to be ported to datacubes, the native one is not working
# to create the world map, simply set local_map=false

#create a global map with rainforest_values to see a general distribution of rainforest worldwide
global_figure = Rainforestlib.build_figure_by_lcc_classes(lc, rainforest_values; local_map = false, set_nan = true)

In [ ]:
using .Rainforestlib

# local is the default, and makes the map as large as the datapoints
# but can me changed by setting 

using Colors, ColorSchemes

custom_colormap = [
    RGB(1.0, 1.0, 1.0),  # Red
    RGB(0.0, 0.0, 1.0),  # Blue
    RGB(0.0, 1.0, 0.0)   # Green
]
#build a map of the rainforest in south america
local_figure = Rainforestlib.build_figure_by_lcc_classes(filtered_dc, all_forest_values; colormap = custom_colormap, colorrange = (0,1), set_nan = true)

In [ ]:
Rainforestlib.build_plots_over_time(filtered_dc, all_forest_values; resolution=(2000, 2000), set_nan = true)

In [ ]:
# build one figure with diffs for each timestep

using .Rainforestlib
using Colors, ColorSchemes
using GLMakie


custom_colormap = [
    RGB(1.0, 1.0, 1.0),  # White
    RGB(0.0, 1.0, 0.0),  # Green
    RGB(0.0, 0.0, 1.0),  # Blue
    RGB(1.0, 0.0, 0.0),  # Red
]

Rainforestlib.build_diff_figures_over_time(filtered_dc, Rainforestlib.LCCSClasses.categories["rainforest"], "imgs"; colormap = custom_colormap, resolution = (3000, 3000))

In [ ]:
#using .Rainforestlib

# local is the default, and makes the map as large as the datapoints
# but can me changed by setting 

searched_values = all_forest_values

using Colors, ColorSchemes

custom_colormap = [
    RGB(1.0, 0.0, 0.0),  # Red
    RGB(0.0, 0.0, 1.0),  # Blue
    RGB(0.0, 1.0, 0.0)   # Green
]


local_figure = Rainforestlib.build_figure_by_lcc_classes(filtered_dc, searched_values; colormap = custom_colormap, colorrange=(1,2))


using CairoMakie

xs = 0:0.5:10
ys = sin.(xs)

points = [scatter!(xs, sin.(xs .* i), color = color)
    for (i, color) in zip(1:length(custom_colormap), custom_colormap)]

# xs = 0:0.1:10
# lins = [lines!(xs, sin.(xs .* i), color = color)
#     for (i, color) in zip(1:3, [:red, :blue, :green])]

# xs = 0:0.1:10
# lins = [lines!(xs, sin.(xs .+ 3v), color = RGBf(v, 0, 1-v)) for v in 0:0.1:0.2]

Legend(
    local_figure[1, 1], 
    points,
    ["$lcss_class" for lcss_class in first(searched_values,3)],
    "Legend",
    tellheight = false,
    tellwidth = false,
    margin = (110, 10, 50, 10),
    halign = :left, valign = :bottom, orientation = :vertical
)

local_figure